# Demo: Multi-echo preprocessing in Neurodesk (fMRIPrep 25.2.3 → tedana)

This is a lean, reproducible demo for **one participant** (`sub-10317`) from OpenNeuro `ds005123`.

Pipeline:
1. DataLad install + `datalad get sub-10317`
2. **Neurodesk module**: `ml fmriprep/25.2.3`
3. Run fMRIPrep with **MNI152NLin6Asym only** + `--me-output-echos`
4. Install `tedana` (no module) and run it on **MNI echo-wise fMRIPrep outputs**


In [ ]:
import os, glob, json, re
from pathlib import Path

# ---- User-set: where to run ----
# If you prefer Neurodesk persistent storage, set PROJ to:
#   /neurodesktop-storage/neurodesktop-storage/ds005123_me_demo
PROJ = Path.home() / "ds005123_me_demo"

SUB = "10317"

BIDS = PROJ / "bids"
FMRIPREP_OUT = PROJ / "derivatives" / "fmriprep-25.2.3"
WORK = PROJ / "work"
TEDANA_OUT = PROJ / "derivatives" / "tedana"

LOGS = PROJ / "logs"
LICENSES = PROJ / "licenses"
TEMPLATEFLOW = PROJ / "templateflow"
MPLCONFIG = PROJ / "mplconfigdir"

for p in [PROJ, BIDS, FMRIPREP_OUT, WORK, TEDANA_OUT, LOGS, LICENSES, TEMPLATEFLOW, MPLCONFIG]:
    p.mkdir(parents=True, exist_ok=True)

# Export for shell commands
os.environ.update({
    "PROJ": str(PROJ),
    "SUB": SUB,
    "BIDS": str(BIDS),
    "FMRIPREP_OUT": str(FMRIPREP_OUT),
    "WORK": str(WORK),
    "TEDANA_OUT": str(TEDANA_OUT),
    "LOGS": str(LOGS),
    "LICENSES": str(LICENSES),
    "TEMPLATEFLOW_HOME": str(TEMPLATEFLOW),
    "MPLCONFIGDIR": str(MPLCONFIG),
})

print("PROJ:", PROJ)
print("BIDS:", BIDS)
print("FMRIPREP_OUT:", FMRIPREP_OUT)
print("TEDANA_OUT:", TEDANA_OUT)


## 1) DataLad: install dataset + get `sub-10317` (quiet-ish)

We log output to files and show only the last few lines.


In [ ]:
# Install into PROJ/bids (if already installed, this is fast)
!bash -lc 'cd "$PROJ" && datalad -l error install -s https://github.com/OpenNeuroDatasets/ds005123.git bids > "$LOGS/datalad_install.log" 2>&1 || true'
!bash -lc 'tail -n 12 "$LOGS/datalad_install.log" || true'

# Download only the target subject
!bash -lc 'cd "$BIDS" && datalad -l error get "sub-$SUB" > "$LOGS/datalad_get_sub-${SUB}.log" 2>&1'
!bash -lc 'tail -n 12 "$LOGS/datalad_get_sub-${SUB}.log"'

# Quick check (should list multi-echo BOLD files)
!bash -lc 'ls "$BIDS/sub-$SUB/func" | head'


## 2) FreeSurfer license for fMRIPrep

We copy `~/.license` into the project so the path is stable.


In [ ]:
!bash -lc 'test -r "$HOME/.license" || (echo "ERROR: FreeSurfer license not found at ~/.license" && exit 1)'
!bash -lc 'cp -f "$HOME/.license" "$LICENSES/fs_license.txt"'
!bash -lc 'ls -l "$LICENSES/fs_license.txt"'


## 3) Run fMRIPrep (Neurodesk module)

Requirements you specified:
- `ml fmriprep/25.2.3`
- `--output-spaces MNI152NLin6Asym`
- `--me-output-echos`
- avoid the FreeSurfer `subjects_dir` crash by ensuring `SUBJECTS_DIR` exists

This cell runs the whole command in **one shell**, so `ml ...` applies to the run.


In [ ]:
!bash -lc 'set -euo pipefail

ml fmriprep/25.2.3
fmriprep --version

# Keep caches local/predictable
mkdir -p "$TEMPLATEFLOW_HOME" "$MPLCONFIGDIR"

# Prevent the subjects_dir TraitError (directory must exist)
export SUBJECTS_DIR="$FMRIPREP_OUT/sourcedata/freesurfer"
mkdir -p "$SUBJECTS_DIR"

# Avoid oversubscription
export OMP_NUM_THREADS=1
export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1

mkdir -p "$FMRIPREP_OUT" "$WORK"

fmriprep "$BIDS" "$FMRIPREP_OUT" participant   --participant-label "$SUB"   --stop-on-first-crash   --skip-bids-validation   --me-output-echos   --output-spaces MNI152NLin6Asym   --fs-no-reconall   --fs-license-file "$LICENSES/fs_license.txt"   -w "$WORK"   --nthreads 14 --omp-nthreads 1 --mem-mb 24000   2>&1 | tee "$LOGS/fmriprep_sub-${SUB}.log"
'


In [ ]:
# Quick fMRIPrep checks
!bash -lc 'ls -l "$FMRIPREP_OUT/sub-$SUB.html"'
!bash -lc 'ls "$FMRIPREP_OUT/sub-$SUB/func" | grep "space-MNI152NLin6Asym" | head'


## 4) Select one run + build tedana inputs (MNI echo-wise outputs)

We automatically pick the first **echo-1** preprocessed file in MNI space and derive:
- the matching echo-wise fMRIPrep files (all echoes)
- the corresponding raw JSON sidecars to extract `EchoTime` values


In [ ]:
from pathlib import Path
import glob, json, re, os

# Pick the first echo-1, MNI-space, preprocessed BOLD file
candidates = sorted(glob.glob(str(Path(os.environ["FMRIPREP_OUT"]) / f"sub-{SUB}/func/*echo-1*space-MNI152NLin6Asym*desc-preproc_bold.nii.gz")))
if not candidates:
    raise RuntimeError("No MNI echo-1 preprocessed files found. Confirm fMRIPrep finished and used --me-output-echos.")

prep_echo1 = Path(candidates[0])
print("Selected fMRIPrep echo-1 file:")
print(" ", prep_echo1)

# Parse into (prefix, suffix) around echo index for processed file naming
# Example: <prefix>_echo-1<suffix> where suffix includes space/desc/etc.
m = re.match(r"^(.*)_echo-\d+(.*)$", prep_echo1.name)
if not m:
    raise RuntimeError(f"Could not parse echo structure from: {prep_echo1.name}")
run_prefix = m.group(1)
proc_suffix = m.group(2)

# Derive raw suffix for BIDS filenames by stripping everything from "_space-" onward,
# then appending "_bold.nii.gz". This handles things like part-mag correctly.
cut = proc_suffix.find("_space-")
if cut == -1:
    raise RuntimeError("Could not locate '_space-' in processed filename suffix.")
raw_suffix = proc_suffix[:cut] + "_bold.nii.gz"  # e.g., _part-mag_bold.nii.gz

print("\nRUN_PREFIX:", run_prefix)
print("PROC_SUFFIX (processed):", proc_suffix[:80] + ("..." if len(proc_suffix) > 80 else ""))
print("RAW_SUFFIX (raw):", raw_suffix)

# Collect echo-wise processed inputs (all echoes), sorted by echo number
prep_glob = str(Path(os.environ["FMRIPREP_OUT"]) / f"sub-{SUB}/func/{run_prefix}_echo-*{proc_suffix}")
prep_echos = sorted(glob.glob(prep_glob))
print("\nProcessed echoes (inputs to tedana):", len(prep_echos))
for p in prep_echos:
    print(" ", Path(p).name)

# Collect raw JSON sidecars to extract TEs
json_glob = str(Path(os.environ["BIDS"]) / f"sub-{SUB}/func/{run_prefix}_echo-*{raw_suffix.replace('_bold.nii.gz','.json')}")
json_paths = sorted(glob.glob(json_glob))
if not json_paths:
    raise RuntimeError("No JSON sidecars found for this run.")

tes = []
print("\nEchoTimes from JSON (seconds):")
for jp in json_paths:
    with open(jp, "r") as f:
        tes.append(float(json.load(f)["EchoTime"]))
    print(" ", Path(jp).name, "->", tes[-1])

# Save for next steps
os.environ["RUN_PREFIX"] = run_prefix
os.environ["PROC_SUFFIX"] = proc_suffix
os.environ["RAW_SUFFIX"] = raw_suffix
os.environ["TE_LIST"] = " ".join(str(x) for x in tes)

print("\nTE_LIST:", os.environ["TE_LIST"])


## 5) Install `tedana` (no module) and run it

Neurodesk does not provide a `tedana` module here, so we install via pip.

To reduce noise:
- install logs go to `logs/pip_tedana.log`
- we print only a short tail


In [ ]:
# Install tedana (user install) and confirm it is on PATH
!bash -lc 'python3 -m pip install --user tedana > "$LOGS/pip_tedana.log" 2>&1'
!bash -lc 'tail -n 12 "$LOGS/pip_tedana.log"'

# Ensure ~/.local/bin is on PATH for subsequent cells
import os
os.environ["PATH"] = str(Path.home() / ".local" / "bin") + ":" + os.environ.get("PATH","")

!bash -lc 'which tedana && tedana --version'


In [ ]:
# Run tedana on the selected run using MNI echo-wise fMRIPrep outputs
!bash -lc 'set -euo pipefail

# Make sure tedana is on PATH inside this shell too
export PATH="$HOME/.local/bin:$PATH"

run_id="${RUN_PREFIX}${RAW_SUFFIX%.nii.gz}"  # readable label (prefix + part-mag)
outdir="$TEDANA_OUT/sub-$SUB/func/${run_id}_space-MNI152NLin6Asym"
mkdir -p "$outdir"

# Build the echo list in a stable (echo-number) order
mapfile -t ECHOS < <(ls "$FMRIPREP_OUT/sub-$SUB/func/${RUN_PREFIX}_echo-*${PROC_SUFFIX}" | sort -V)

tedana   -d "${ECHOS[@]}"   -e ${TE_LIST}   --convention bids   --out-dir "$outdir"   --prefix "${run_id}_space-MNI152NLin6Asym"   --fittype curvefit   --overwrite
'


In [ ]:
# Quick tedana check
!bash -lc 'ls -lh "$TEDANA_OUT/sub-$SUB/func" | head'
